# Define Environment

In [ ]:
class Environment():
    
    '''
    This wrapper works as an RL-Environment for the Jetson Nano.
    Arguments are: 
        model  -> pytorch model trained in Unity.
        angles -> list of angles required to measure
        FOV    -> How wide the camera lens is. Default = 160
    '''
    
    def __init__(self, model, angles, robot, FOV=160):
        self.previous_readings = {x:0 for x in angles} 
        self.angles = angles
        self.model = model
        self.FOV = FOV
        self.robot = robot
        
        
    def calculate_angle_and_phase(self):
        keypoints, image, counts, objects, peaks = execute2({'new': camera.value})
        angle, keypoints = calculate_angle(WIDTH, keypoints, self.FOV)
        phase = self.calculate_phase(keypoints)
        return phase, angle
    
    
    def step(tensor):
        turn, move = tensor.cpu()
        
    
    def stop(self):
        self.robot.stop()

        
    def step_forward(self):
        self.robot.forward(0.4)
        time.sleep(0.5)
        self.robot.stop()

        
    def step_backward(self):
        self.robot.backward(0.4)
        time.sleep(0.5)
        self.robot.stop()

        
    def step_left(self):
        self.robot.left(0.3)
        time.sleep(0.5)
        self.robot.stop()

        
    def step_right(self):
        self.robot.right(0.3)
        time.sleep(0.5)
        self.robot.stop()
    
    
    def calculate_phase(self, keypoints):
        phase = estimating_phase(keypoints)
        return phase
    
    
    def read_lidar(self):
        self.previous_readings = read_lidar_wrapper(self.angles,self.previous_readings)
        
    
    def observe(self):
        self.read_lidar()
        phase, angle = self.calculate_angle_and_phase()
        
        if isinstance(angle, dict):
            angles = list(angle.values())
        
        else: angles = [-1]
        
        observation = phase + angle + list(self.previous_readings.values())
        return observation
    
    
    def sample_action(self, observation):
        observation = torch.Tensor(observation).cuda()
        hidden,_ = self.model.network_body(vis_inputs=[0],vec_inputs=[observation])
        distribution = self.model.distribution(hidden)
        action = distribution.sample()
        return action
    
    
    def step(self,action):
        action = action.cpu().detach().numpy()[0]
        
        speed_move, speed_turn = float(abs(action[0])), float(abs(action[1]))
        
        # backward
        if action[0] < 0:
            robot.backward(speed_move)
            time.sleep(0.5)
            robot.stop()
            
        # forward
        else:
            robot.forward(speed_move)
            time.sleep(0.5)
            robot.stop()
            
        time.sleep(0.2)
            
        # turn left
        if action[1] < 0:
            robot.left(speed_turn)
            time.sleep(0.3)
            robot.stop()
            
        # turn right
        else:
            robot.right(speed_turn)
            time.sleep(0.3)
            robot.stop()
